In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['okay.xlsx']


In [14]:
X = pd.read_excel("../input/okay.xlsx")

In [15]:
y_new = pd.DataFrame(data = X['dcfdx'])

In [ ]:
p = pd.DataFrame(data=X['projid'].value_counts())
ids1 = []
ids2_chk = []
val = []
arr = p.index
for i in arr:
    if p['projid'][i] == 1:
        ids1.append(i)
ids1

In [ ]:
z = pd.DataFrame(data=X.groupby(['projid']).dcfdx.mean())
#zz = z.index[0]
#X['dcfdx']
indices = []
ids = []
a = z.index
ct = 0
for i in z['dcfdx']:
    if int(i) == i:
        indices.append(ct)
    #print(i)
    ct+=1
for i in indices:
    ids.append(a[i])
ids

In [16]:
# feature-select
final_features = ['bun', 'ca', 'chlstrl', 'cl', 'crn', 'gfr_mdrs', 'glucose', 'hba1c',
       'hdlchlstrl', 'hemoglbn', 'k', 'ldlchlstrl', 'mcv', 'na', 'platelet',
       'rbc', 'tga', 'tsh', 'wbc', 'age_at_visit', 'bmi', 'hypertension_cum',
       'cancer_cum', 'stroke_cum', 'analgesic_rx','antianxiety_rx',
       'antibiotic_rx', 'lipid_lowering_rx','mental_health_rx','thyroid_cum',
       'mental_health_rx', 'neurologic_rx', 'antihyp_rx', 'antireflux_rx',
       'laxative_rx', 'statin_rx', 'antipsychotic_rx', 'depression_rx',
       'alternative_rx', 'vitamin_rx']
drop_cols_list = list(set(X.columns) - set(final_features))
print(len(drop_cols_list))

66


In [17]:
X_new = X.drop(drop_cols_list, axis=1)
X_new.shape

(25570, 39)

In [8]:
X_new.head()

,bun,ca,chlstrl,cl,crn,gfr_mdrs,glucose,hba1c,hdlchlstrl,hemoglbn,k,ldlchlstrl,mcv,na,platelet,rbc,tga,tsh,wbc,age_at_visit,bmi,hypertension_cum,cancer_cum,thyroid_cum,stroke_cum,analgesic_rx,antibiotic_rx,lipid_lowering_rx,mental_health_rx,neurologic_rx,antihyp_rx,antireflux_rx,laxative_rx,statin_rx,antianxiety_rx,antipsychotic_rx,depression_rx,alternative_rx,vitamin_rx
0,13,9.5,235,96,0.85,68.300013,92,5.6,79,12.9,5.0,133,101.7,136,246,3.73,114,1.67,4.8,79.969884,26.831775,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1
1,19,9.4,185,103,0.96,63.911495,97,5.8,58,13.2,4.3,97,92.7,140,220,4.24,123,2.05,6.2,81.081451,27.615143,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
2,13,9.4,205,102,0.98,57.803255,77,5.5,76,12.6,4.2,112,99.6,139,175,3.69,85,2.47,5.1,81.612594,26.975260,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,13,9.4,230,93,0.89,64.444732,85,5.5,79,13.1,4.3,134,99.5,130,226,3.78,86,2.14,4.5,82.595483,26.401319,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,14,9.2,210,99,0.82,70.657593,92,6.1,69,12.8,4.5,119,100.1,136,202,3.88,109,2.43,5.3,83.594798,35.144482,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [18]:
# Scale - 
p = X_new.loc[:, 'bun':'bmi']
q = X_new.loc[:, 'hypertension_cum':'vitamin_rx']

In [19]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
p_df = pd.DataFrame(scaler.fit_transform(p))
p_df.columns = p.columns

ct = 0
z = X_new['age_at_visit']
for i in p_df['age_at_visit']:
    p_df['age_at_visit'][ct] = int(z[ct])
    ct+=1
p_df.head(3)

,bun,ca,chlstrl,cl,crn,gfr_mdrs,glucose,hba1c,hdlchlstrl,hemoglbn,k,ldlchlstrl,mcv,na,platelet,rbc,tga,tsh,wbc,age_at_visit,bmi
0,0.072917,0.534247,0.517134,0.236842,0.089750,0.057109,0.161290,0.121951,0.420382,0.489209,0.611111,0.440860,0.575323,0.40625,0.245327,0.346470,0.112965,0.012162,0.018980,79.0,0.329496
1,0.135417,0.520548,0.361371,0.421053,0.102505,0.053154,0.172043,0.138211,0.286624,0.510791,0.416667,0.311828,0.446198,0.53125,0.214953,0.430213,0.124519,0.014945,0.026573,81.0,0.344051
2,0.072917,0.520548,0.423676,0.394737,0.104824,0.047649,0.129032,0.113821,0.401274,0.467626,0.388889,0.365591,0.545194,0.50000,0.162383,0.339901,0.075738,0.018020,0.020607,81.0,0.332162


In [20]:
X_new = pd.concat([p_df,q], ignore_index=False, sort=False, axis=1)
X_new.head(2)

,bun,ca,chlstrl,cl,crn,gfr_mdrs,glucose,hba1c,hdlchlstrl,hemoglbn,k,ldlchlstrl,mcv,na,platelet,rbc,tga,tsh,wbc,age_at_visit,bmi,hypertension_cum,cancer_cum,thyroid_cum,stroke_cum,analgesic_rx,antibiotic_rx,lipid_lowering_rx,mental_health_rx,neurologic_rx,antihyp_rx,antireflux_rx,laxative_rx,statin_rx,antianxiety_rx,antipsychotic_rx,depression_rx,alternative_rx,vitamin_rx
0,0.072917,0.534247,0.517134,0.236842,0.089750,0.057109,0.161290,0.121951,0.420382,0.489209,0.611111,0.440860,0.575323,0.40625,0.245327,0.346470,0.112965,0.012162,0.018980,79.0,0.329496,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1
1,0.135417,0.520548,0.361371,0.421053,0.102505,0.053154,0.172043,0.138211,0.286624,0.510791,0.416667,0.311828,0.446198,0.53125,0.214953,0.430213,0.124519,0.014945,0.026573,81.0,0.344051,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1


In [21]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.3, random_state=42)
x_train1, x_test1, y_train1, y_test1 = train_test_split(x_train, y_train, test_size=0.3, random_state=42)

In [22]:
import lightgbm as lgb
from bayes_opt import BayesianOptimization
from sklearn.metrics import roc_auc_score

def lgb_evalx(num_leaves, feature_fraction, bagging_fraction, max_depth, lambda_l1,
             lambda_l2, min_split_gain, min_child_weight):
    params = {'application': 'binary', 'num_iterations': 4000, 'learning_rate': 0.05,
              'early_stopping_round': 100, 'metric': auc}
    params["num_leaves"] = round(num_leaves)
    params["feature_fraction"] = max(min(feature_fraction, 1), 0)
    params["bagging_fraction"] = max(min(bagging_fraction, 1), 0)
    params["max_depth"] = round(max_depth)
    params['lambda_l1'] = max(lambda_l1, 0)
    params['lambda_l2'] = max(lambda_l2, 0)
    params["min_split_gain"] = min_split_gain
    params["min_child_weight"] = min_child_weight
    cv_results = lgb.cv(params, train_data, nfold=n_folds, seed=random_seed, stratified=True, 
                        verbose_eval = 200, metrics=['auc'])
    return max(cv_results['auc-mean'])

In [23]:
lgbBo = BayesianOptimization(lgb_evalx, {'num_leaves': (10,50),
                                        'feature_fraction': (0.1, 0.9),
                                        'bagging_fraction': (0.8, 1),
                                        'max_depth': (5, 9),
                                        'lambda_l1': (0, 5),
                                        'lambda_l2': (0, 3),
                                        'min_split_gain': (0.001, 0.1),
                                        'min_child_weight': (5, 50)}, random_state=42)

In [26]:
import warnings
warnings.filterwarnings('ignore')

In [39]:
def bayes_parameter_opt_lgb(XX, yy, init_round=15, opt_round=25, n_folds=5, random_seed=42,
                           n_estimators=10000, learning_rate=0.05, output_process=False):
    train_data = lgb.Dataset(data=XX, label=yy)
    def lgb_eval(num_leaves, feature_fraction, bagging_fraction, max_depth, lambda_l1, lambda_l2, min_split_gain, min_child_weight):
        params = {'application':'binary','num_iterations': n_estimators, 'learning_rate':learning_rate,
                  'early_stopping_round':100, 'metric':'auc'}
        params["num_leaves"] = int(round(num_leaves))
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
        params['max_depth'] = int(round(max_depth))
        params['lambda_l1'] = max(lambda_l1, 0)
        params['lambda_l2'] = max(lambda_l2, 0)
        params['min_split_gain'] = min_split_gain
        params['min_child_weight'] = min_child_weight
        cv_result = lgb.cv(params, train_data, nfold=n_folds, seed=random_seed,
                           stratified=True, verbose_eval=200, metrics=['auc'])
        return max(cv_result['auc-mean'])
    # range 
    lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (15, 45),
                                            'feature_fraction': (0.1, 0.9),
                                            'bagging_fraction': (0.8, 1),
                                            'max_depth': (5, 8.99),
                                            'lambda_l1': (0, 5),
                                            'lambda_l2': (0, 3),
                                            'min_split_gain': (0.001, 0.1),
                                            'min_child_weight': (5, 50)}, random_state=0)
    lgbBO.maximize(init_points=init_round, n_iter=opt_round)
    return lgbBO.max

optimal_params = bayes_parameter_opt_lgb(x_train1, y_train1, init_round=20, opt_round=100,
                                         n_folds=3, random_seed=6, n_estimators=2000, learning_rate=0.05)


|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_sp... | num_le... |
-------------------------------------------------------------------------------------------------------------------------
|  1        |  1.0      |  0.9098   |  0.6722   |  3.014    |  1.635    |  6.69     |  34.07    |  0.04432  |  41.75    |
|  2        |  1.0      |  0.9927   |  0.4068   |  3.959    |  1.587    |  7.266    |  46.65    |  0.008033 |  17.61    |
|  3        |  1.0      |  0.804    |  0.7661   |  3.891    |  2.61     |  8.905    |  40.96    |  0.04669  |  38.42    |
|  4        |  1.0      |  0.8237   |  0.6119   |  0.7168   |  2.834    |  7.082    |  23.66    |  0.02719  |  38.23    |
|  5        |  1.0      |  0.8912   |  0.5547   |  0.09395  |  1.853    |  7.442    |  32.76    |  0.09443  |  35.45    |
|  6        |  1.0      |  0.8719   |  0.4496   |  3.488    |  0.1807   |  7.66     |  35.18    |  0.02183  |  18.87    |
|  7        |  1.0      

|  41       |  1.0      |  0.9034   |  0.2023   |  4.53     |  2.651    |  5.279    |  5.114    |  0.05461  |  15.03    |
|  42       |  1.0      |  0.9914   |  0.2102   |  3.199    |  0.2062   |  5.022    |  49.75    |  0.01911  |  15.25    |
|  43       |  1.0      |  0.8497   |  0.5541   |  0.1797   |  0.8598   |  6.216    |  5.04     |  0.07291  |  44.63    |
|  44       |  1.0      |  0.9667   |  0.2996   |  0.2136   |  2.824    |  5.204    |  49.93    |  0.01708  |  15.33    |
|  45       |  1.0      |  0.8019   |  0.8429   |  0.12     |  1.303    |  8.269    |  5.605    |  0.0656   |  15.05    |
|  46       |  1.0      |  0.937    |  0.6248   |  0.5371   |  0.6801   |  5.717    |  49.76    |  0.09274  |  44.75    |
|  47       |  1.0      |  0.8035   |  0.8584   |  1.628    |  2.932    |  8.476    |  5.039    |  0.0767   |  15.02    |
|  48       |  1.0      |  0.8269   |  0.7211   |  0.1234   |  1.012    |  8.707    |  49.78    |  0.048    |  44.96    |
|  49       |  1.0      

|  82       |  1.0      |  0.9608   |  0.6567   |  0.02341  |  0.9054   |  6.707    |  49.78    |  0.004344 |  44.65    |
|  83       |  1.0      |  0.9878   |  0.3463   |  0.4258   |  2.556    |  5.633    |  5.001    |  0.06663  |  15.06    |
|  84       |  1.0      |  0.9138   |  0.7689   |  0.242    |  0.3132   |  8.975    |  49.82    |  0.05207  |  15.22    |
|  85       |  1.0      |  0.8776   |  0.1874   |  0.9218   |  2.628    |  5.175    |  5.162    |  0.00928  |  44.77    |
|  86       |  1.0      |  0.9272   |  0.4364   |  1.699    |  2.753    |  5.013    |  5.036    |  0.01561  |  15.2     |
|  87       |  1.0      |  0.9234   |  0.7534   |  2.782    |  0.351    |  7.734    |  5.125    |  0.05127  |  44.96    |
|  88       |  1.0      |  0.8778   |  0.4561   |  4.131    |  2.59     |  7.692    |  49.76    |  0.0937   |  16.12    |
|  89       |  1.0      |  0.9151   |  0.7236   |  3.096    |  2.964    |  6.53     |  49.41    |  0.003492 |  43.38    |
|  90       |  1.0      

In [40]:
print(optimal_params)

{'target': 1.0, 'params': {'bagging_fraction': 0.909762700785465, 'feature_fraction': 0.6721514930979355, 'lambda_l1': 3.0138168803582195, 'lambda_l2': 1.6346495489906907, 'max_depth': 6.690382649362229, 'min_child_weight': 34.065235087999525, 'min_split_gain': 0.04432113391500656, 'num_leaves': 41.753190023462395}}
